In [56]:
import os
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

# 🗓 Define the injury report date range
start_date = "2024-07-01"
end_date = "2025-06-30"

# 📁 Create subfolder to save all injury files
OUTPUT_DIR = "injured_players"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 🏀 NBA team abbreviations used on the site
NBA_TEAMS = [
    "hawks", "celtics", "nets", "hornets", "bulls", "cavaliers", "mavericks", "nuggets",
    "pistons", "warriors", "rockets", "pacers", "clippers", "lakers", "grizzlies", "heat",
    "bucks", "timberwolves", "pelicans", "knicks", "thunder", "magic", "76ers", "suns",
    "blazers", "kings", "spurs", "raptors", "jazz", "wizards"
]

# 🌐 Base URL
BASE_URL = "https://www.prosportstransactions.com/basketball/Search/SearchResults.php"

def parse_table(html):
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find("table", class_="datatable center")
    if not table:
        return pd.DataFrame()

    headers = ["Date", "Team", "Acquired", "Relinquished", "Notes"]
    rows = []
    for tr in table.find_all("tr")[2:]:
        cells = [td.get_text(" ", strip=True) for td in tr.find_all("td")]
        if cells and len(cells) == len(headers):
            rows.append(cells)

    return pd.DataFrame(rows, columns=headers)

def scrape_team_injuries(team, begin_date, end_date):
    all_data = []
    start = 0
    session = requests.Session()
    session.headers.update({
        "User-Agent": "Mozilla/5.0",
        "Accept": "text/html",
    })

    while True:
        params = {
            "Player": "",
            "Team": team,
            "BeginDate": begin_date,
            "EndDate": end_date,
            "InjuriesChkBx": "yes",
            "Submit": "Search",
            "start": str(start)
        }

        response = session.get(BASE_URL, params=params)
        if response.status_code != 200:
            break

        df = parse_table(response.text)
        if df.empty:
            break

        all_data.append(df)
        start += 25
        time.sleep(1)

    return pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()

def clean_player(name):
    if pd.isna(name):
        return None
    return (
        name.encode("ascii", errors="ignore")
        .decode()
        .replace("â€¢", "")
        .replace("•", "")
        .replace("\xa0", " ")
        .strip()
    )

def build_daily_calendar(df):
    for col in ["Acquired", "Relinquished"]:
        df[col] = df[col].apply(clean_player)

    df["Date"] = pd.to_datetime(df["Date"])

    events = []
    for _, row in df.iterrows():
        date = row["Date"]
        out_player = clean_player(row["Relinquished"])
        in_player = clean_player(row["Acquired"])
        if out_player:
            events.append({"player": out_player, "status": "OUT", "date": date})
        if in_player:
            events.append({"player": in_player, "status": "IN", "date": date})

    events_df = pd.DataFrame(events).sort_values("date").reset_index(drop=True)
    if events_df.empty:
        return pd.DataFrame()

    min_date = events_df["date"].min()
    max_date = events_df["date"].max()
    all_dates = pd.date_range(start=min_date, end=max_date, freq="D")

    player_status = {}
    calendar_rows = []

    for day in all_dates:
        day_events = events_df[events_df["date"] == day]
        for _, event in day_events.iterrows():
            if event["status"] == "OUT":
                player_status[event["player"]] = True
            elif event["status"] == "IN":
                player_status[event["player"]] = False

        out_today = sorted([p for p, status in player_status.items() if status])
        calendar_rows.append({
            "Date": day.strftime("%Y-%m-%d"),
            "OutPlayers": ", ".join(out_today)
        })

    return pd.DataFrame(calendar_rows)

# 🌀 Run for all NBA teams
today_str = datetime.today().strftime("%Y-%m-%d")

for team in NBA_TEAMS:
    print(f"\n🏀 Processing team: {team.upper()}")

    df_injuries = scrape_team_injuries(team, start_date, end_date)

    if not df_injuries.empty:
        injury_path = os.path.join(OUTPUT_DIR, f"{team}_injuries_{today_str}.csv")
        df_injuries.to_csv(injury_path, index=False)
        print(f"✅ Saved injury report to: {injury_path}")

        calendar_df = build_daily_calendar(df_injuries)
        if not calendar_df.empty:
            calendar_path = os.path.join(OUTPUT_DIR, f"{team}_injury_calendar_{today_str}.csv")
            calendar_df.to_csv(calendar_path, index=False)
            print(f"📆 Daily calendar saved to: {calendar_path}")
        else:
            print("⚠️ No calendar events generated.")
    else:
        print(f"❌ No injury data found for {team}.")



🏀 Processing team: HAWKS
✅ Saved injury report to: injured_players\hawks_injuries_2025-04-09.csv
📆 Daily calendar saved to: injured_players\hawks_injury_calendar_2025-04-09.csv

🏀 Processing team: CELTICS
✅ Saved injury report to: injured_players\celtics_injuries_2025-04-09.csv
📆 Daily calendar saved to: injured_players\celtics_injury_calendar_2025-04-09.csv

🏀 Processing team: NETS
✅ Saved injury report to: injured_players\nets_injuries_2025-04-09.csv
📆 Daily calendar saved to: injured_players\nets_injury_calendar_2025-04-09.csv

🏀 Processing team: HORNETS
✅ Saved injury report to: injured_players\hornets_injuries_2025-04-09.csv
📆 Daily calendar saved to: injured_players\hornets_injury_calendar_2025-04-09.csv

🏀 Processing team: BULLS
✅ Saved injury report to: injured_players\bulls_injuries_2025-04-09.csv
📆 Daily calendar saved to: injured_players\bulls_injury_calendar_2025-04-09.csv

🏀 Processing team: CAVALIERS
✅ Saved injury report to: injured_players\cavaliers_injuries_2025-04-09